<a href="https://colab.research.google.com/github/secondfo/Car-price-prediction/blob/main/submission_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

hackathon_qualification_path = kagglehub.competition_download('hackathon-qualification')

print('Data source import complete.')


In [ ]:
import os
print(os.listdir("/kaggle/input/hackathon-qualification/archive"))



In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from catboost import CatBoostRegressor


In [ ]:
train = pd.read_csv("/kaggle/input/hackathon-qualification/archive/train.csv")
test = pd.read_csv("/kaggle/input/hackathon-qualification/archive/test.csv")

print(train.shape)
print(test.shape)
train.head()


In [ ]:
y = train["price"]

X = train.drop(["id", "price"], axis=1)
X_test = test.drop("id", axis=1)


In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [ ]:
categorical_cols = X_train.select_dtypes(include=["object"]).columns

for col in categorical_cols:
    X_train[col] = X_train[col].fillna("missing").astype(str)
    X_valid[col] = X_valid[col].fillna("missing").astype(str)
    X_test[col] = X_test[col].fillna("missing").astype(str)

for col in X_train.select_dtypes(include=[np.number]).columns:
    X_train[col] = X_train[col].fillna(X_train[col].median())
    X_valid[col] = X_valid[col].fillna(X_train[col].median())
    X_test[col] = X_test[col].fillna(X_train[col].median())



In [ ]:
cat_feature_indices = [X_train.columns.get_loc(col) for col in categorical_cols]

cat_model = CatBoostRegressor(
    iterations=1000,
    depth=8,
    learning_rate=0.1,
    loss_function="RMSE",
    cat_features=cat_feature_indices,
    verbose=200
)


In [ ]:
cat_model.fit(
    X_train, y_train,
    eval_set=(X_valid, y_valid),
   # cat_features=categorical_cols,
    use_best_model=True
)


y_pred = cat_model.predict(X_valid)

# RMSE
rmse = mean_squared_error(y_valid, y_pred, squared=False)
print(f"Validation RMSE: ", rmse)


In [ ]:
final_preds = cat_model.predict(X_test)

submission = pd.DataFrame({
    "id": test["id"],
    "price": final_preds
})

submission.to_csv("submission.csv", index=False)
submission.head()
print("Final submission")